In [2]:
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score
from tqdm import tqdm
from scipy.stats import entropy
import math
import random

torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

train_df = pd.read_csv('train.csv')
val_df = pd.read_csv('validation.csv')
test_df = pd.read_csv('test.csv')
T_TTA_test_df = pd.read_csv('T_TTA_enhanced_test.csv')
R_TTA_test_df = pd.read_csv('R_TTA_enhanced_test.csv')
MSTTA_test_df = pd.read_csv('MSTTA_enhanced_test.csv')

class SentimentDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        sentence = self.dataframe.iloc[index]['sentence']
        label = self.dataframe.iloc[index]['label']
        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True,
        )
        return {
            'sentence_text': sentence,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

def create_data_loader(df, tokenizer, max_len, batch_size,shuffle):
    ds = SentimentDataset(
        dataframe=df,
        tokenizer=tokenizer,
        max_len=max_len
    )
    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=0,
        pin_memory=True,
        shuffle=shuffle
    )

PRE_TRAINED_MODEL_NAME = 'roberta-large'
tokenizer = RobertaTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
model = RobertaForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME, num_labels=5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
model = model.to(device)

max_len = 128
batch_size = 32

train_data_loader = create_data_loader(train_df, tokenizer, max_len, batch_size,shuffle=True)
val_data_loader = create_data_loader(val_df, tokenizer, max_len, batch_size,shuffle=False)
test_data_loader = create_data_loader(test_df, tokenizer, max_len, batch_size,shuffle=False)
T_TTA_test_data_loader = create_data_loader(T_TTA_test_df, tokenizer, max_len, batch_size,shuffle=False)
R_TTA_test_data_loader = create_data_loader(R_TTA_test_df, tokenizer, max_len, batch_size,shuffle=False)
MSTTA_test_data_loader= create_data_loader(MSTTA_test_df, tokenizer, max_len, batch_size,shuffle=False)


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifie

True


In [2]:
for param in model.roberta.parameters():
    param.requires_grad = False
    
for param in model.roberta.encoder.layer[-4:].parameters():
    param.requires_grad = True

for param in model.classifier.parameters():
    param.requires_grad = True

for name, param in model.named_parameters():
    print(f"{name}: {param.requires_grad}")
    
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5, correct_bias=False)
total_steps = len(train_data_loader) * 10  

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = torch.nn.CrossEntropyLoss().to(device)

def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0

    for d in tqdm(data_loader, desc="Training"):
        input_ids = d['input_ids'].to(device)
        attention_mask = d['attention_mask'].to(device)
        labels = d['label'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        _, preds = torch.max(outputs.logits, dim=1)
        loss = loss_fn(outputs.logits, labels)

        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for d in tqdm(data_loader, desc="Evaluating"):
            input_ids = d['input_ids'].to(device)
            attention_mask = d['attention_mask'].to(device)
            labels = d['label'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, preds = torch.max(outputs.logits, dim=1)
            loss = loss_fn(outputs.logits, labels)

            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

    return correct_predictions.double() / n_examples, np.mean(losses)

best_loss = np.inf
patience = 3 
counter = 0  

for epoch in range(10):
    print(f'Epoch {epoch + 1}/10')
    print('-' * 50)

    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
        scheduler,
        len(train_df)
    )

    print(f'Train loss {train_loss} accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model,
        val_data_loader,
        loss_fn,
        device,
        len(val_df)
    )

    print(f'Val   loss {val_loss} accuracy {val_acc}')

    if val_loss < best_loss:
        torch.save(model.state_dict(), 'roberta_large_Compare_model.pth')
        best_loss = val_loss
        counter = 0  
    else:
        counter += 

    if counter >= patience:
        print("Early stopping triggered")
        break


roberta.embeddings.word_embeddings.weight: False
roberta.embeddings.position_embeddings.weight: False
roberta.embeddings.token_type_embeddings.weight: False
roberta.embeddings.LayerNorm.weight: False
roberta.embeddings.LayerNorm.bias: False
roberta.encoder.layer.0.attention.self.query.weight: False
roberta.encoder.layer.0.attention.self.query.bias: False
roberta.encoder.layer.0.attention.self.key.weight: False
roberta.encoder.layer.0.attention.self.key.bias: False
roberta.encoder.layer.0.attention.self.value.weight: False
roberta.encoder.layer.0.attention.self.value.bias: False
roberta.encoder.layer.0.attention.output.dense.weight: False
roberta.encoder.layer.0.attention.output.dense.bias: False
roberta.encoder.layer.0.attention.output.LayerNorm.weight: False
roberta.encoder.layer.0.attention.output.LayerNorm.bias: False
roberta.encoder.layer.0.intermediate.dense.weight: False
roberta.encoder.layer.0.intermediate.dense.bias: False
roberta.encoder.layer.0.output.dense.weight: False
robe

Training: 100%|██████████████████████████████████████████████████████████████████████| 267/267 [01:54<00:00,  2.34it/s]


Train loss 1.284204612956958 accuracy 0.42743445692883897


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 35/35 [00:10<00:00,  3.27it/s]


Val   loss 1.1090264746120997 accuracy 0.5177111716621253
Epoch 2/10
--------------------------------------------------


...

Val   loss 1.095794197491237 accuracy 0.5258855585831063
Epoch 5/10
--------------------------------------------------


Training: 100%|██████████████████████████████████████████████████████████████████████| 267/267 [01:59<00:00,  2.23it/s]


Train loss 0.8791782460855634 accuracy 0.6197331460674157


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 35/35 [00:10<00:00,  3.29it/s]

Val   loss 1.126639723777771 accuracy 0.5122615803814714
Early stopping triggered


In [4]:
def predict_probabilities(model, data_loader,device):
    model.eval()
    sentence_probs = []

    with torch.no_grad():
        for d in tqdm(data_loader, desc="Predicting"):
            input_ids = d['input_ids'].to(device)
            attention_mask = d['attention_mask'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
            sentence_probs.extend(probs.cpu().numpy())

    return np.array(sentence_probs)

model.load_state_dict(torch.load('roberta_large_Compare_model.pth'))
true_labels = test_df['label'].values

simple_preds = []

with torch.no_grad():
    for d in tqdm(test_data_loader, desc="Predicting Simple"):
        input_ids = d['input_ids'].to(device)
        attention_mask = d['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        simple_preds.extend(preds.cpu().numpy())

simple_accuracy = accuracy_score(true_labels, simple_preds)
print(f'Simple Accuracy: {simple_accuracy}')

def calculate_uncertainty_weights(predictions):
    mean_prediction = np.mean(predictions, axis=0)
    variance = np.var(predictions, axis=0) + 1e-10
    uncertainties = []

    for prediction in predictions:
        diff_square = (prediction - mean_prediction) ** 2
        temp_var = 0.5 * np.log(2 * math.pi * variance)
        temp_max = np.maximum(0, temp_var)
        final_uncertainty = temp_max + (diff_square / (math.sqrt(3) * variance))
        uncertainties.append(final_uncertainty)

    weights = 1 / (np.array(uncertainties) + 1e-10)
    return weights / weights.sum(axis=0)  

def get_grouped_probs(data_loader, model,device):
    sentence_probs = predict_probabilities(model, data_loader,device)
    return [sentence_probs[i:i+3] for i in range(0, len(sentence_probs), 3)]

def compute_weighted_predictions(grouped_probs, weight_type="average"):
    weighted_probs = []
    for group in grouped_probs:
        if weight_type == "average":
            weights = np.array([1/3, 1/3, 1/3])
        elif weight_type == "uncertainty":
            weights = calculate_uncertainty_weights(group)
        else:
            raise ValueError(f"Unknown weight type: {weight_type}")
        weighted_probs.append(np.average(group, axis=0, weights=weights))
    return np.array(weighted_probs)

test_simple_probs = predict_probabilities(model, test_data_loader,device)
T_TTA_grouped_probs = get_grouped_probs(T_TTA_test_data_loader, model,device)
R_TTA_grouped_probs = get_grouped_probs(R_TTA_test_data_loader, model,device)
MSTTA_grouped_probs = get_grouped_probs(MSTTA_test_data_loader, model,device)

simple_preds = np.argmax(test_simple_probs, axis=1)
simple_accuracy = accuracy_score(true_labels, simple_preds)
simple_precision = precision_score(true_labels, simple_preds, average='weighted')
simple_recall = recall_score(true_labels, simple_preds, average='weighted')
simple_f1 = f1_score(true_labels, simple_preds, average='weighted')
print(f"Simple Accuracy: {simple_accuracy:.4f}, Precision: {simple_precision:.4f}, Recall: {simple_recall:.4f}, F1: {simple_f1:.4f}")

weight_types = ["average",  "uncertainty"]
results = {}

for weight_type in weight_types:
    T_TTA_weighted_probs = compute_weighted_predictions(T_TTA_grouped_probs, weight_type=weight_type)
    R_TTA_weighted_probs = compute_weighted_predictions(R_TTA_grouped_probs, weight_type=weight_type)
    MSTTA_weighted_probs = compute_weighted_predictions(MSTTA_grouped_probs, weight_type=weight_type)

    T_TTA_preds = np.argmax(T_TTA_weighted_probs, axis=1)
    R_TTA_preds = np.argmax(R_TTA_weighted_probs, axis=1)
    MSTTA_preds = np.argmax(MSTTA_weighted_probs, axis=1)
    
    true_labels = test_df['label'].values
    results[f"T_TTA {weight_type}"] = {
        "accuracy": accuracy_score(true_labels, T_TTA_preds),
        "f1": f1_score(true_labels, T_TTA_preds, average='weighted'),
        "precision": precision_score(true_labels, T_TTA_preds, average='weighted'),
        "recall": recall_score(true_labels, T_TTA_preds, average='weighted')
    }
    results[f"R_TTA {weight_type}"] = {
        "accuracy": accuracy_score(true_labels, R_TTA_preds),
        "f1": f1_score(true_labels, R_TTA_preds, average='weighted'),
        "precision": precision_score(true_labels, R_TTA_preds, average='weighted'),
        "recall": recall_score(true_labels, R_TTA_preds, average='weighted')
    }
    results[f"MSTTA {weight_type}"] = {
        "accuracy": accuracy_score(true_labels, MSTTA_preds),
        "f1": f1_score(true_labels, MSTTA_preds, average='weighted'),
        "precision": precision_score(true_labels, MSTTA_preds, average='weighted'),
        "recall": recall_score(true_labels, MSTTA_preds, average='weighted')
    }

for method, metrics in results.items():
    print(f"{method} - Accuracy: {metrics['accuracy']:.4f}, Precision: {metrics['precision']:.4f}, Recall: {metrics['recall']:.4f}, F1: {metrics['f1']:.4f}")


Predicting Simple: 100%|███████████████████████████████████████████████████████████████| 70/70 [00:21<00:00,  3.31it/s]


Simple Accuracy: 0.5855203619909503


Predicting: 100%|████████████████████████████████████████████████████████████████████| 208/208 [01:05<00:00,  3.17it/s]


Simple Accuracy: 0.5855, Precision: 0.5785, Recall: 0.5855, F1: 0.5682
T_TTA average - Accuracy: 0.5810, Precision: 0.5761, Recall: 0.5810, F1: 0.5633
R_TTA average - Accuracy: 0.5738, Precision: 0.5702, Recall: 0.5738, F1: 0.5556
MSTTA average - Accuracy: 0.5819, Precision: 0.5798, Recall: 0.5819, F1: 0.5642
T_TTA uncertainty - Accuracy: 0.5792, Precision: 0.5733, Recall: 0.5792, F1: 0.5615
R_TTA uncertainty - Accuracy: 0.5842, Precision: 0.5793, Recall: 0.5842, F1: 0.5665
MSTTA uncertainty - Accuracy: 0.5900, Precision: 0.5860, Recall: 0.5900, F1: 0.5731
